In [2]:
import re
from urllib import *
from html import unescape
import requests
import json 

In [3]:
from urllib import error
import requests
import time
import bs4
def download(method,url,param=None,data=None, timeout=1, maxretries=3):
    #res = requests.request(method,url,param=param,data=data,headers=headers)
    
    try:
        resp=requests.request(method, url,params=param,data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500<=e.response.status_code<600 and maxretries>0:
            print(maxretries)
            
            download(method,url,param,data,timeout,maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp
    

In [49]:
#오늘은 시스템적으로 www를 계속 탐험하게 만들 것임 부산물은 html 이다.

In [50]:
#사이트의 영향력은 인바운드와 아웃바운드로 알 수 있다.

In [5]:
#a라는 태그에서 attr중에 href를 가지고 온다.(html)

In [4]:
headers ={"user-agent":"#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
html=download("get","http://example.webscraping.com/places/default/index")
from bs4 import BeautifulSoup

In [5]:
from bs4 import BeautifulSoup

In [7]:
dom = BeautifulSoup(html.text,"lxml")
[_["href"] for _ in dom.find_all("a") if len(_["href"])>3]

['/places/default/user/register?_next=/places/default/index',
 '/places/default/user/login?_next=/places/default/index',
 '/places/default/index',
 '/places/default/search',
 '/places/default/view/Afghanistan-1',
 '/places/default/view/Aland-Islands-2',
 '/places/default/view/Albania-3',
 '/places/default/view/Algeria-4',
 '/places/default/view/American-Samoa-5',
 '/places/default/view/Andorra-6',
 '/places/default/view/Angola-7',
 '/places/default/view/Anguilla-8',
 '/places/default/view/Antarctica-9',
 '/places/default/view/Antigua-and-Barbuda-10',
 '/places/default/index/1']

In [42]:
url = "https://search.naver.com/search.naver?where=nexearch&sm=tab_jum&query=%EB%B0%95%EB%B3%B4%EC%98%81"
html = download("get", url)
dom = BeautifulSoup(html.text,"lxml")
[_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title") if checkBlog(_["href"])]
# if checkBlog(_["href"])

['https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950',
 'https://blog.naver.com/js2y86?Redirect=Log&logNo=221530397718',
 'https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195']

In [45]:

def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    ## naver iframe html안에 html 또들어있어서
    if len(dom.select("#mainFrame")) < 1:  ## 메인이랑, 첫페이지만 없음
        return []
    
    seed = requests.compat.urljoin(seed,dom.select("#mainFrame")[0]["src"])
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
 #   print(requests.compat.urljoin(seed,dom.select("#mainFame")[0]["src"]))

    return  [requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") if  _.has_attr("href") and len(_["href"]) >3 and checkBlog(requests.compat.urljoin(seed, _["href"]))]

In [22]:

def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
   
    return  [(requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") if seed[1] < 2 and _.has_attr("href") and len(_["href"])  and checkBlog(requests.compat.urljoin(seed[0], _["href"]))]

SyntaxError: invalid syntax (<ipython-input-22-551f1aabff19>, line 6)

In [76]:
queue = list()
queue.extend([(_["href"],1) for _ in dom.select("#blogColl a.f_link_b")if checkBlog(_["href"])])
seen=list()
print(queue)
while queue:
    baseURL = queue.pop(0)
    print(baseURL)
    seen.append(baseURL)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue:{0} Seen:{1}".format(len(queue),len(seen)))        
    #queue.extend(linkList)

[('http://adam24eve.tistory.com/858', 1), ('http://cscs0717.tistory.com/27', 1), ('http://listup.tistory.com/248', 1), ('http://34344541.tistory.com/43', 1)]
('http://adam24eve.tistory.com/858', 1)
Queue:48 Seen:1
('http://cscs0717.tistory.com/27', 1)
Queue:77 Seen:2
('http://listup.tistory.com/248', 1)
Queue:103 Seen:3
('http://34344541.tistory.com/43', 1)
Queue:126 Seen:4
('http://adam24eve.tistory.com/', 2)
Queue:125 Seen:5
('http://adam24eve.tistory.com/category', 2)
Queue:124 Seen:6
('http://adam24eve.tistory.com/category/Cosmos', 2)
Queue:123 Seen:7
('http://adam24eve.tistory.com/category/Mystery%20world', 2)
Queue:122 Seen:8
('http://adam24eve.tistory.com/category/Star', 2)
Queue:121 Seen:9
('http://adam24eve.tistory.com/category/Political%20', 2)
Queue:120 Seen:10
('http://adam24eve.tistory.com/category/It-game', 2)
Queue:119 Seen:11
('http://adam24eve.tistory.com/guestbook', 2)
Queue:118 Seen:12
('http://adam24eve.tistory.com/858', 2)
Queue:117 Seen:13
('https://adam24eve.tist

Queue:8 Seen:122
('http://34344541.tistory.com/39', 2)
Queue:7 Seen:123
('http://34344541.tistory.com/1', 2)
Queue:6 Seen:124
('http://34344541.tistory.com/54', 2)
Queue:5 Seen:125
('http://34344541.tistory.com/53', 2)
Queue:4 Seen:126
('http://34344541.tistory.com/52', 2)
Queue:3 Seen:127
('http://34344541.tistory.com/51', 2)
Queue:2 Seen:128
('http://34344541.tistory.com/43', 2)
Queue:1 Seen:129
('https://34344541.tistory.com/manage', 2)
Queue:0 Seen:130


In [ ]:
#linkList에 중복되는 애들이 있을 수 있다.
#urljoin의 역할이 무엇인가??

In [28]:
requests.compat.urlparse("https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950")

ParseResult(scheme='https', netloc='blog.naver.com', path='/imagetech1', params='', query='Redirect=Log&logNo=221530358950', fragment='')

In [63]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")

In [54]:
html=download("get","https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&q=%EB%B0%95%EB%B3%B4%EC%98%81")
dom=BeautifulSoup(html.text,"lxml")

In [55]:
[_["href"] for _ in dom.select("#blogColl a.f_link_b") if checkBlog(_["href"])]

['http://adam24eve.tistory.com/858',
 'http://cscs0717.tistory.com/27',
 'http://listup.tistory.com/248',
 'http://34344541.tistory.com/43']

In [71]:
ll=[(1,"a"),(2,"b"),(3,"c"),(1,"d"),(1,"e"),(1,"f")]

In [72]:
ll[0]

(1, 'a')

In [73]:
ll[0][0]

1

In [77]:
#db만들기 1.id 2.url 3.첫액세스 date 4. seen인지 아닌지 체크 ->최소 4개의 컬럼을 생성

In [78]:
#특정 url을 가진애만 쓰고 싶을 때는 sql로 url을 건드려야한다. 
#string연산은 느리므로 테이블을 하나 더 생성한다
#t1 id path seen data t2 id netloc date
#t1과 t2를 조인하면 하나의 url이된다. t2에 특정 데이터만 써주면 된다.
#url = netloc + path + params

In [13]:
import sqlite3

In [14]:
con = sqlite3.connect("botfin.db")
cur = con.cursor()

In [52]:
def parseURL(seed):
    html = download('get', seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    return [requests.compat.urljoin(seed, _['href']) for _ in dom.find_all('a') 
            if _.has_attr('href') and not _['href'].startswith('#')]

In [15]:
cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table2_id INTEGER NOT NULL,
        path TEXT NOT NULL,
        param TEXT,
        depth INTEGER NOT NULL,
        inbound INTEGER NOT NULL,
        seen BOOLEAN DEFAULT FALSE NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );

""")
con.commit()

In [16]:
headers ={"user-agent":"#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
html = download("get","https://www.google.com/search?q=%EB%B0%95%EB%B3%B4%EC%98%81")
dom = BeautifulSoup(html.text,"lxml")

for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
    _urlparse=requests.compat.urlparse(href)
    netloc="://".join(_urlparse[:2])
    cur.execute("SELECT id FROM table2 WHERE netloc =? LIMIT 0,1",[netloc])
    netlocID =cur.fetchone()
    if not cur.fetchone():
        cur.execute("INSERT INTO table2(netloc) VALUES(?)",[netloc])
        con.commit()
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1",[netloc])
        netlocID =cur.fetchone()
        
    cur.execute("INSERT INTO table1(table2_id,path,param,depth,inbound) VALUES(?,?,?,?,?)",[netlocID[0], _urlparse[2], _urlparse[4],1,0])   
    print(cur.lastrowid, href)
    con.commit()    

#seed url -> db insert db select limit 0,1

1 https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81
2 https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%EC%9D%98_%EC%9E%91%ED%92%88_%EB%AA%A9%EB%A1%9D
3 https://news.joins.com/article/22895953
4 https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81
5 https://movie.daum.net/person/main?personId=105244
6 http://m.cafe.daum.net/parkboyoungfd
7 https://www.msn.com/ko-kr/entertainment/news/%EB%B0%95%EB%B3%B4%EC%98%81-tvn-%EB%93%9C%EB%9D%BC%EB%A7%88-%EC%96%B4%EB%B9%84%EC%8A%A4-%EC%97%AC%EC%A3%BC%EC%9D%B8%EA%B3%B5/ar-BBLZzVh
8 https://www.msn.com/ko-kr/entertainment/news/%EB%84%88%EC%9D%98-%EA%B2%B0%ED%98%BC%EC%8B%9D-%E2%86%92-%EC%96%B4%EB%B9%84%EC%8A%A4-%EB%B0%95%EB%B3%B4%EC%98%81-%EA%B7%BC%ED%99%A9-%EC%82%B4%ED%8E%B4%EB%B3%B4%EB%8B%88%E2%80%A6%E2%80%9C%EC%97%AC%EC%A0%84%ED%9E%88-%EB%BD%80%EB%B8%94%EB%A6%AC%E2%80%9D/ar-BBUOmqm


In [33]:
i = 0
while True:
    # seen=FASLE인 것의 정보를 가져온다.
    # 먼저 들어간 애가 먼저 나오게 한다. 
    cur.execute("""
        SELECT table1.id, table2.netloc, table1.path, table1.param , table1.depth, table2.id
        FROM table1 
        JOIN table2 
            ON table1.table2_id = table2.id 
        WHERE table1.seen='FALSE' and table1.depth < 3
        ORDER BY table1.date ASC 
        LIMIT 0,1;
    """)
    seed = cur.fetchone()
    #seed가 없으면 종료(i가 1000을 넘으면 종료)
    if not seed or i > 100:
        break;
    i+=1
    print(i)
        
   # DB에 seen=FALSE인 것을 불러와서 볼 것이니까 TRUE로 전환
    cur.execute("""
        UPDATE table1
        SET seen='TRUE'
        WHERE id=?
    """, [seed[0]])
    con.commit()
    
    # 보려는 주소 조합하기
    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    
#     # 보고 있는 곳의 링크들을 가져와서 저장한다.
#     for href in parseURL(baseURL):
#         _urlparse = requests.compat.urlparse(href)
#         netloc = "://".join(_urlparse[:2])
#         cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
#         netlocID = cur.fetchone()
#         if not netlocID:
#             cur.execute("INSERT INTO table2(netloc) VALUES(?)",[netloc])
#             con.commit()
        
#             cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
#             netlocID = cur.fetchone()
    
#         cur.execute("INSERT INTO table1(table2_id, path, param, depth, inbound) VALUES(?, ?, ?, ?, ?)",[netlocID[0], _urlparse[2], _urlparse[4], seed[4]+1, seed[5]])
#         con.commit()

1
2
3


In [41]:
cur.close()
con.close()

In [147]:
cur.execute("SELECT inbound,count(inbound) FROM table1 WHERE table2_id=9 GROUP BY inbound")

In [148]:
cur.fetchall()

[(0, 1), (2, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 9)]

In [149]:
cur.execute("SELECT count(inbound) FROM table1 WHERE inbound=2")
cur.fetchall()

[(2,)]

In [23]:

def parseURL(seed):
    html = download("get",seed[0])
    dom = BeautifulSoup(html.text, "lxml")
   
    return  [(requests.compat.urljoin(seed[0], _["href"]) ,seed[1]+1) for _ in dom.find_all("a") if seed[1] < 2 and _.has_attr("href") and len(_["href"])  and checkBlog(requests.compat.urljoin(seed[0], _["href"]))]